In [10]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

/home/rakesh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# working with Kaggle set first, then to rt-polarity to check acc, then transcript sentences

train_data=pd.read_csv('/research/home/rakesh/notebooks/dennybritz/keras_example_dataset/train.csv')
test_data=pd.read_csv('/research/home/rakesh/notebooks/dennybritz/keras_example_dataset/test.csv')

In [6]:
print(train_data.shape,test_data.shape)
print(train_data.isnull().sum())
print(test_data.isnull().sum())

(19579, 3) (8392, 2)
id        0
text      0
author    0
dtype: int64
id      0
text    0
dtype: int64


In [8]:
# converting author names to cateogories

authors=train_data.author.unique()
dic={}
for i,author in enumerate(authors):
    dic[author]=i
labels=train_data.author.apply(lambda x:dic[x])

In [20]:
labels.unique()

array([0, 1, 2])

In [9]:
# split the train data itself into train and validate - before running on eval

val_data=train_data.sample(frac=0.2,random_state=200)
train_data=train_data.drop(val_data.index)

#### Pre processing the text

In [11]:
texts=train_data.text

In [12]:
NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid=tokenizer.texts_to_sequences(val_data.text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 23090 unique tokens.


In [13]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
y_train = to_categorical(np.asarray(labels[train_data.index]))
y_val = to_categorical(np.asarray(labels[val_data.index]))
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (15663, 861) (3916, 861)
Shape of label train and validation tensor: (15663, 3) (3916, 3)


#### Keras model

In [14]:
# generate own embedding - word2vec can also be tried here

from keras.layers import Embedding

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)

embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM)

In [15]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers

sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [16]:
## training

adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, y_train, batch_size=1000, epochs=10, verbose=1, validation_data=(X_val, y_val),
         callbacks=callbacks)  # starts training

Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 393s 25ms/step - loss: 1.1507 - acc: 0.4113 - val_loss: 1.1243 - val_acc: 0.4242
Epoch 2/10
15663/15663 [==============================] - 448s 29ms/step - loss: 1.0937 - acc: 0.4986 - val_loss: 1.0705 - val_acc: 0.5105
Epoch 3/10
15663/15663 [==============================] - 333s 21ms/step - loss: 1.0111 - acc: 0.6167 - val_loss: 0.9822 - val_acc: 0.6964
Epoch 4/10
15663/15663 [==============================] - 321s 20ms/step - loss: 0.8811 - acc: 0.7630 - val_loss: 0.8845 - val_acc: 0.7383
Epoch 5/10
15663/15663 [==============================] - 301s 19ms/step - loss: 0.7492 - acc: 0.8221 - val_loss: 0.8019 - val_acc: 0.7837
Epoch 6/10
15663/15663 [==============================] - 311s 20ms/step - loss: 0.6488 - acc: 0.8658 - val_loss: 0.7540 - val_acc: 0.7909
Epoch 7/10
15663/15663 [==============================] - 306s 20ms/step - loss: 0.5777 - acc: 0.8981 - val_loss: 0.7

In [18]:
1+1

2